In [30]:
import pandas as pd
import re
from time import time

import sys
reload(sys)
sys.setdefaultencoding('utf8')
pd.options.display.max_columns=100

In [31]:
!pwd

In [32]:
# !ls /home/saya/notebook/twitter/

In [33]:
df=pd.read_csv('/home/saya/notebook/twitter/twitter_basement.csv')
df.shape

(479066, 7)

In [34]:
df.head()

created_at  \
0  Tue Sep 12 17:17:51 +0000 2017   
1  Tue Sep 12 17:17:54 +0000 2017   
2  Tue Sep 12 17:17:56 +0000 2017   
3  Tue Sep 12 17:17:58 +0000 2017   
4  Tue Sep 12 17:17:58 +0000 2017   

                                                text      user.name  \
0  RT @WiseBasementSys: It's #VideoGamesDay - Hav...  Ben Gab.io ��������   
1  RT @the8asement: 2-PLAYER CHALLENGE on Eventid...     MellyLumpi   
2  RT @aparnapkin: How can I be a misogynist if I...   Mike Leatham   
3  RT @DothTheDoth: My kink is being lost in an a...     Persephone   
4  2k, Spades, Dominoes, we in a whole different ...         ICB ������   

                  user.created_at  user.followers_count  user.friends_count  \
0  Mon Jun 30 05:50:45 +0000 2014                1297.0              2489.0   
1  Sat Apr 02 20:56:23 +0000 2016                   5.0                25.0   
2  Sun Aug 16 19:08:53 +0000 2015                  34.0               282.0   
3  Wed May 03 10:56:26 +0000 2017                  40.0                93.0   
4  Sat Apr 10 15:26:03 +0000 2010                1869.0               925.0   

    user.id_str  
0  2.595851e+09  
1  7.163687e+17  
2  3.426488e+09  
3  8.597233e+17  
4  1.315358e+08

In [35]:
test=df['text'][0]

In [36]:
print test

In [37]:
def clean(test):
    return re.sub(r'@\S+|#|RT|\n|http\S+','',str(test)).strip()

In [38]:
a = df['text'][29]
a

'Sundays Open House. #Realtor #Pool #Basement #Acre #RuralArea https://t.co/ELNiaVcvQs'

In [39]:
content = df['text'].apply(clean)

In [40]:
len(content)

479066

### topic modeling

In [41]:
import nltk
import string
from nltk.stem.wordnet import WordNetLemmatizer

# nltk.download('stopwords')
from nltk.corpus import stopwords 
stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation) 
lemmatize = WordNetLemmatizer()


In [42]:
stop = pd.read_fwf('stop.txt',names=['words'])
stopwords.update(set(stop['words'].values))
# print (list(stopwords)[:10])
stopwords.update(set(['dont','lot','wa','im','ha','doe','wa', 'able',]))

In [43]:
def cleaning(article):
    article = str(article)
    zero = "".join(i for i in article if i not in punctuation)
    
    one = " ".join([i for i in zero.lower().split() if i not in stopwords])
    
    three = " ".join(lemmatize.lemmatize(i) for i in one.split())
    return three

In [44]:
all_content=content.copy()
apple = all_content

In [45]:
# nltk.download('wordnet')
start = time()

ap_text = apple.apply(cleaning)
ap_text_list = [i.split() for i in ap_text]
print (len(ap_text_list))

('used: {:.2f}s'.format(time()-start))

'used: 51.37s'

In [46]:
ap_text[:10]

0    videogamesday dreamed converting basement arca...
1           2player challenge eventide island basement
2                         misogynist live mom basement
3    kink lost abandoned house amp hearing tune pia...
4    2k spade domino mode join godsc jewel inc toni...
5                         misogynist live mom basement
6        yo basement story pinegroves dope found base���
7    mini child programming basement sweatshop produce
8    videogamesday dreamed converting basement arca...
9                            mind laundry basement def
Name: text, dtype: object

In [47]:
from gensim.models.phrases import Phraser, Phrases

start = time()
phs = Phrases(ap_text_list)
('used: {:.2f}s'.format(time()-start))

'used: 6.06s'

In [48]:
bi_gram = Phraser(phs)
start = time()

new_bi_list = [bi_gram[i] for i in ap_text_list]

('used: {:.2f}s'.format(time()-start))

'used: 11.54s'

In [49]:
start = time()

phs3=Phrases(new_bi_list)
tri_gram=Phraser(phs3)

('used: {:.2f}s'.format(time()-start))

'used: 7.95s'

In [50]:
start = time()

new_tri_list = [tri_gram[i] for i in new_bi_list]

('used: {:.2f}s'.format(time()-start))

'used: 7.86s'

In [51]:
len(new_tri_list)

479066

In [52]:
new_tri_list[0]

[u'videogamesday_dreamed_converting',
 u'basement',
 u'arcade_youre_retrogames\u2026']

In [53]:
make_dic = dict()
for i,j in zip(range(len(new_tri_list)), new_tri_list):
    make_dic[i]=j

In [54]:
without_stop = pd.Series(make_dic)
without_stop.head()

0    [videogamesday_dreamed_converting, basement, a...
1     [2player, challenge, eventide, island, basement]
2                      [misogynist_live_mom, basement]
3    [kink_lost_abandoned_house, amp_hearing_tune_p...
4    [2k_spade_domino_mode, join_godsc_jewel_inc, t...
dtype: object

In [55]:
ap_text_list = list(without_stop.values)

In [56]:
len(ap_text_list)

479066

In [57]:
ap_text_list[100:103]

[[u'broadcaster',
  u'criticism',
  u'stagebut',
  u'guy',
  u'eating',
  u'soup',
  u'mom',
  u'basement',
  u'look',
  u'mirror\u2026'],
 [u'nice', u'basement'],
 [u'buddy', u'basement', u'fuck']]

### start building

In [58]:
# Importing Gensim
import gensim
from gensim import corpora

from time import time
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO,
                   filename='twitter_10_8.log',filemode='w')

In [59]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
dictionary = corpora.Dictionary(ap_text_list)
dictionary.save('twitter_10_8.dict')
(dictionary)

In [60]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in ap_text_list]
corpora.MmCorpus.serialize('twitter_10_8.mm', doc_term_matrix)

(len(doc_term_matrix))
(doc_term_matrix[100])

[(2, 1),
 (32, 1),
 (66, 1),
 (332, 1),
 (333, 1),
 (334, 1),
 (335, 1),
 (336, 1),
 (337, 1),
 (338, 1)]

In [70]:
start = time()
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, 
#                passes=5
              )
'used: {:.2f}s'.format(time()-start)

'used: 384.46s'

In [71]:
ldamodel.print_topics()

[(0,
  u'0.141*"basement" + 0.016*"dweller" + 0.014*"mommy" + 0.013*"maga" + 0.012*"fought_lift_flag_thst" + 0.012*"dweller_cound_burn_trump" + 0.011*"thats" + 0.009*"check" + 0.008*"as" + 0.007*"2"'),
 (1,
  u'0.115*"basement" + 0.053*"gonna" + 0.032*"lord_watchingbecause_that\u2019s_exactly" + 0.030*"blessing" + 0.025*"god_sent_reason_deny" + 0.015*"i\u2019m" + 0.012*"huge" + 0.011*"friend" + 0.008*"don\u2019t" + 0.008*"survivalist_spends_45_filling"'),
 (2,
  u'0.127*"basement" + 0.038*"ima_steal_money_church" + 0.014*"love" + 0.013*"day" + 0.012*"party" + 0.011*"shit" + 0.010*"fuck" + 0.010*"lord_watchingbecause_that\u2019s_exactly" + 0.008*"story" + 0.008*"cant"'),
 (3,
  u'0.152*"basement" + 0.016*"amp" + 0.015*"parent" + 0.014*"life" + 0.014*"house" + 0.014*"live" + 0.013*"wrong_people" + 0.012*"29_don\u2026" + 0.012*"people_unattractive_mom" + 0.012*"\U0001f644_easy_call"'),
 (4,
  u'0.382*"basement" + 0.042*"fuck_girl_boyfriend_easily" + 0.042*"wtf_create" + 0.042*"ricans" + 0

In [72]:
ldamodel.save('twitter_10_8.model')

### load saved model , plotting

In [73]:
from gensim.models import LdaModel
loading = LdaModel.load('twitter_10_8.model')

In [74]:
import pyLDAvis.gensim
import gensim
from time import time
pyLDAvis.enable_notebook()

In [75]:
d = gensim.corpora.Dictionary.load('twitter_10_8.dict')
c = gensim.corpora.MmCorpus('twitter_10_8.mm')
lda = gensim.models.LdaModel.load('twitter_10_8.model')

In [76]:
start = time()

data = pyLDAvis.gensim.prepare(lda, c, d)

('used: {:.2f}s'.format(time()-start))

'used: 193.55s'

In [77]:
pyLDAvis.save_html(data,'twitter_10_8.html')


In [78]:
data

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      13.567952        1       1 -0.176783  0.016928
4      12.673745        1       2 -0.178441  0.064553
6      11.672439        1       3 -0.178311  0.016824
0      10.574853        1       4 -0.136105 -0.005282
8      10.225155        1       5  0.242503  0.067277
5       9.213789        1       6  0.243558  0.120733
2       9.065800        1       7 -0.123729 -0.006377
7       8.827994        1       8  0.157421 -0.401286
1       7.953969        1       9 -0.107909 -0.002213
9       6.224303        1      10  0.257798  0.128842, topic_info=      Category           Freq                                 Term  \
term                                                                 
92011  Default  229017.000000                             basement   
67058  Default   45210.000000               embracing_blessing_god   
10324  Default   11886.000000                               period   
24436  Default    9735.000000           fuck_girl_boyfriend_easily   
42646  Default    9734.000000                           wtf_create   
2189   Default    7679.000000                                gonna   
29759  Default   10012.000000                               ricans   
73184  Default    9551.000000                food_���_decided_puerto   
12962  Default    8960.000000           survivalist_filled_massive   
50946  Default    6384.000000               ima_steal_money_church   
39709  Default    5267.000000                          house_music   
18337  Default    6295.000000  lord_watchingbecause_that���s_exactly   
70165  Default    7613.000000                                  mom   
13574  Default    4354.000000                             blessing   
34796  Default    3891.000000                       pup_spent_life   
14008  Default    3698.000000             wild_cat_spent_someone���s   
8256   Default    3698.000000              ���_can���t_believe_they���re   
44651  Default    3833.000000                         ���_ton_friend   
42685  Default    3634.000000            actually_feeling_grass_a���   
26798  Default    3949.000000                                 2011   
25980  Default    3633.000000                 god_sent_reason_deny   
67977  Default    4921.000000                               living   
38933  Default    5567.000000                                  amp   
56062  Default    3177.000000                           housemusic   
78936  Default    3047.000000                                   t���   
12556  Default    3518.000000                                 life   
31241  Default    3047.000000                              dweller   
15910  Default    3426.000000                                 live   
26218  Default    2397.000000                          spread_love   
13850  Default    2397.000000        hello_creative_project_called   
...        ...            ...                                  ...   
60751  Topic10     390.835247                                sweet   
21198  Topic10     365.530152                             starting   
16031  Topic10     352.633351                                 hall   
73685  Topic10     321.760149     knocked_loose_turnstile_movement   
79795  Topic10     321.760149                      story_neck_deep   
14399  Topic10     321.760149         turnover_citizen_title_fight   
6578   Topic10     321.760149                           amp_tyler���   
99630  Topic10     293.242402                         living_mommy   
7274   Topic10     272.721284                                 tour   
84109  Topic10     261.464708                                 pick   
66047  Topic10     257.672133                               ground   
88741  Topic10     252.024606                             brooklyn   
89345  Topic10     248.506697                                 tied   
75916  Topic10     244.384961                            attention   